In [92]:
from dash import Dash, dash_table, dcc, html
from dash.dependencies import Input, Output
import pandas as pd
from flask import Flask
from Librerias_SAP import SAP_GUI, Funtions
import pandas as pd
import re
from collections import OrderedDict
from dash import dash_table as dt
from dash import dcc
from dash import html
from dash.dependencies import Input
from dash.dependencies import Output
import numpy as np
import dash_bootstrap_components as dbc
from datetime import datetime, timedelta

In [93]:
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password

Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][0]
password=Keys["Password"][0]
# Initialize session
session=SAP_GUI.SessionSAP(user,password)

In [94]:
Defaul_Column_Pedidos_dia=['Documento', 'GTr', 'Denomin.', 'ClVt', 'Denominación', 'Solic.',
       'Creado el', 'Fecha doc.', 'Pedido', 'Func.', 'Responsab', 'Creado',
       'OrgVt', 'CDis', 'Se', 'OfVta', 'GVen', 'Mon.', 'Valor neto']

In [95]:
# PEDIDOSEXITO


def Search_VA05N(Transsaccion,session,provision,variant):
    session.StartTransaction(Transsaccion)
    session.findById("wnd[0]/tbar[1]/btn[17]").press()
    session.findById("wnd[1]/usr/txtV-LOW").text = provision
    session.findById("wnd[1]/usr/txtENAME-LOW").text = variant
    session.findById("wnd[1]/usr/txtV-LOW").caretPosition = 8
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    return(session)

In [96]:
Transsaccion='va05n'
provision='PEDIDOSEXITO'
variant='JESPINOSAP'

Search_VA05N(Transsaccion,session,provision,variant)

<COMObject <unknown>>

In [97]:
Name_VA05N="Pedidos_diarios"
Ruta_Exito_ZSD010_ZVMI="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_VA05N,session,Ruta_Exito_ZSD010_ZVMI)

In [98]:
Pedidos_VN05N=pd.read_csv(Ruta_Exito_ZSD010_ZVMI+Name_VA05N+".txt",delimiter="\t",skiprows=1)
Pedidos_VN05N=Funtions.Clean_Columns(Pedidos_VN05N)
Pedidos_VN05N=Funtions.default_column(Defaul_Column_Pedidos_dia,Pedidos_VN05N)
#Elimino pedidos con valores nulos
Pedidos_VN05N=Pedidos_VN05N[Pedidos_VN05N['Pedido'].notnull()]

In [99]:
Pedidos_VN05N['Pedido']

0       0020647408
1       0020647394
2       0020647407
3       0020647408
4       0020647392
           ...    
5508    0068022812
5510    0408123275
5526      PPFR8618
5527    0029067961
5528    0157102894
Name: Pedido, Length: 5510, dtype: object

In [100]:
Agenda=["85","20","146","149","50","138","45"]

Pedidos_VN05N['Pedido']=Funtions.Estandarizo_Pedidos(Pedidos_VN05N['Pedido'])
Pedidos_VN05N['Pedido']=Funtions.complete_pedidos(Pedidos_VN05N['Pedido'],Agenda)


In [101]:
Lunes=["0085"]


buscar=Funtions.Search_Agenda_Exito(Pedidos_VN05N['Pedido'],Lunes)
filtro_cencosub=Pedidos_VN05N[Pedidos_VN05N['Pedido'].isin(buscar)]

In [102]:
filtro_cencosub['Pedido'].value_counts()

0085769642    62
0085763459    61
0085768255    60
0085769564    60
0085768075    60
              ..
0085764469     1
0085763426     1
0085763428     1
0085769648     1
0085770620     1
Name: Pedido, Length: 145, dtype: int64

In [106]:
filtro_cencosub['Pedido'].drop_duplicates().value_counts()

0085761010    1
0085768263    1
0085768267    1
0085768261    1
0085768259    1
             ..
0085764471    1
0085764470    1
0085764463    1
0085764490    1
0085770620    1
Name: Pedido, Length: 145, dtype: int64

In [13]:
session.findById("wnd[0]/usr/btn%_SO_DOC_E_%_APP_%-VALU_PUSH").press()

com_error: (-2147352567, 'Ocurrió una excepción.', (619, 'SAP Frontend Server', 'The control could not be found by id.', 'C:\\Program Files (x86)\\SAP\\FrontEnd\\SAPgui\\sapfront.HLP', 393215, 0), None)

In [104]:
def Search_VN05N(Transsaccion,Series,session):      #(column Dataframe)
    """
    Transsaccion: Transsacion a buscar
    Series: Columna del dataframe que quiero copiar
    session: session del usuario
    provision: disposicion de interes
    """
    session.StartTransaction(Transsaccion)
    Series=Series.to_clipboard(index=False, header=False)
    session.findById("wnd[0]/usr/btn%_SO_BSTKD_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/tbar[1]/btn[8]").press()


In [105]:
Search_VN05N('zsd79',filtro_cencosub['Pedido'].drop_duplicates(),session)

In [107]:

session.StartTransaction('zsd79')
Series=filtro_cencosub['Pedido'].to_clipboard(index=False, header=False)
session.findById("wnd[0]/usr/btn%_SO_BSTKD_%_APP_%-VALU_PUSH").press()
session.findById("wnd[1]/tbar[0]/btn[24]").press()
session.findById("wnd[1]/tbar[0]/btn[8]").press()
session.findById("wnd[0]/tbar[1]/btn[8]").press()
table=session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell")

# Nota no tiene entrega generada si Pedido Concluido Rojo y entrega conlcuida Verda

In [125]:
table.SelectColumn("LFGSK")
table.SelectColumn("WBSTK")

In [117]:
#Columna Pedido concluido 1
#table.GetCellValue(2,"LFGSK")
table.SelectColumn("LFGSK")
table.SelectColumn("BUCHK")
#Columna entregas concluidas
#Columna entregas
#table.SelectColumn("ZAD")
#Columna entregas concluidas
table.SelectColumn("WBSTK")

In [89]:
table.SelectColumn("LFGSK")

In [126]:
session.findById("wnd[0]/mbar/menu[1]/menu[3]").select() 
session.findById("wnd[1]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/ctxt%%DYN001-LOW").text = "@0A@"
session.findById("wnd[1]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/ctxt%%DYN002-LOW").text = "@08@" 
session.findById("wnd[1]/tbar[0]/btn[0]").press()

In [128]:
#Columna Pedido concluido 1
table.GetCellValue(2,"LFGSK")

#Columna entregas concluidas
table.SelectColumn("WBSTK")

In [129]:
table.GetCellValue(2,"LFGSK")

'@0A@'

In [142]:
table.GetCellValue(2,"BSTNK")   #Nº Pedido cliente
table.GetCellValue(2,"AUART")   #Clase Orden
table.GetCellValue(2,"AUDAT")   #PrimFecEnt
table.GetCellValue(2,"VDATU")   #ÚltEntrega

'06.07.2023'

In [152]:
Row=table.RowCount
for i in range(0,Row):
    Pedido_cliente,Clase_Orden,PrimFecEnt,ÚltEntrega=table.GetCellValue(i,"BSTNK"),table.GetCellValue(2,"AUART"),table.GetCellValue(2,"AUDAT"),table.GetCellValue(2,"VDATU") 
    print(Pedido_cliente,Clase_Orden,PrimFecEnt,ÚltEntrega)

0085770424 ZVMI 29.06.2023 06.07.2023
0085770425 ZVMI 29.06.2023 06.07.2023
0085770426 ZVMI 29.06.2023 06.07.2023
0085770427 ZVMI 29.06.2023 06.07.2023
0085770428 ZVMI 29.06.2023 06.07.2023
0085770429 ZVMI 29.06.2023 06.07.2023
0085770430 ZVMI 29.06.2023 06.07.2023
0085770431 ZVMI 29.06.2023 06.07.2023
0085770432 ZVMI 29.06.2023 06.07.2023
0085770433 ZVMI 29.06.2023 06.07.2023
0085770589 ZVMI 29.06.2023 06.07.2023
0085770620 ZVMI 29.06.2023 06.07.2023


In [74]:
"@0A@       Filtro Rojo"      "WBSTK"   "BUCHK"
#Columna entregas concluidas
table.SelectColumn("BUCHK")


In [67]:
#Columna entregas
table.GetCellValue(2,"ZAD")
"LFGSK"

'LFGSK'

In [75]:
from time import sleep

In [79]:
sleep(1)
session.findById("wnd[0]/mbar/menu[0]/menu[3]/menu[2]").select()
sleep(3)
session.findById("wnd[1]/usr/subSUBSCREEN_STEPLOOP:SAPLSPO5:0150/sub:SAPLSPO5:0150/radSPOPLI-SELFLAG[1,0]").select()
sleep(3)

In [17]:
def Export_TXT(Name,session,Ruta=None):
    try:
        session.findById("wnd[0]/mbar/menu[0]/menu[3]/menu[2]").select()
    except:
        pass
    finally:
        session.findById("wnd[1]/usr/subSUBSCREEN_STEPLOOP:SAPLSPO5:0150/sub:SAPLSPO5:0150/radSPOPLI-SELFLAG[1,0]").select()
        session.findById("wnd[1]/tbar[0]/btn[0]").press()
        if Ruta==None:
            session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
        else:
            session.findById("wnd[1]/usr/ctxtDY_PATH").text = Ruta
        session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = str(Name) + ".txt"
        session.findById("wnd[1]/usr/ctxtDY_FILE_ENCODING").text = "4310"
        session.findById("wnd[1]/tbar[0]/btn[11]").press()

In [18]:
Name_VA05N="Avisos_despacho"
Ruta_Exito_ZSD010_ZVMI="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
Export_TXT(Name_VA05N,session,Ruta_Exito_ZSD010_ZVMI)

In [97]:
pd.read_csv(Ruta_Exito_ZSD010_ZVMI+Name_VA05N+".txt",delimiter='\t',skiprows=1)[['Fecha doc.', 'Unnamed: 8', 'FePrefEnt.',
       'Unnamed: 10', 'Pedido Concluido', 'Entregas Contab.',
       'Fact. Gen-Cont.', 'Aviso D.', 'Fec. AD', 'Unnamed: 16', 'Reg. AD',
       'F.Re. AD', 'Unnamed: 19', 'Borra. AD', 'F. Borr AD']]

,Fecha doc.,Unnamed: 8,FePrefEnt.,Unnamed: 10,Pedido Concluido,Entregas Contab.,Fact. Gen-Cont.,Aviso D.,Fec. AD,Unnamed: 16,Reg. AD,F.Re. AD,Unnamed: 19,Borra. AD,F. Borr AD
0,NaN,14.06.2023,NaN,17.06.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,14.06.2023,NaN,17.06.2023,NaN,NaN,NaN,NaN,NaN,16.06.2023,NaN,NaN,NaN,NaN,NaN
2,NaN,14.06.2023,NaN,17.06.2023,NaN,NaN,NaN,NaN,NaN,16.06.2023,NaN,NaN,NaN,NaN,NaN
3,NaN,14.06.2023,NaN,17.06.2023,NaN,NaN,NaN,NaN,NaN,16.06.2023,NaN,NaN,NaN,NaN,NaN
4,NaN,14.06.2023,NaN,17.06.2023,NaN,NaN,NaN,NaN,NaN,16.06.2023,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,NaN,28.06.2023,NaN,06.07.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,NaN,28.06.2023,NaN,01.07.2023,NaN,NaN,NaN,NaN,NaN,20.03.2019,NaN,NaN,NaN,NaN,NaN
152,NaN,28.06.2023,NaN,01.07.2023,NaN,NaN,NaN,NaN,NaN,20.03.2019,NaN,NaN,NaN,NaN,NaN
153,NaN,28.06.2023,NaN,08.07.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
prueba = pd.read_html("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\Prueba.htm")